In [0]:
%pip install PyAstronomy
import pandas as pd
from PyAstronomy import pyasl
from IPython.display import clear_output
clear_output(wait=True)
import numpy as np

     |████████████████████████████████| 675kB 31.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/c7/a3/53/9ffbaec17aa68617537651f8c3b9c9d0db767a36dfc6d7e55a
Successfully built PyAstronomy


In [0]:
#Mounting the Google drive for data input
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive
/gdrive


In [0]:
df = pd.read_table("/gdrive/My Drive/Colab Notebooks/Catalina_Vars.tbl", delim_whitespace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [0]:
# conversion of RA and Dec from hh:mm:ss to degree using coordsSexaToDeg and rounding off to 3 decimals for comparison
df["ra"]=None
df["dec"]=None
for i in range(len(df)):
  df["ra"][i], df["dec"][i] = pyasl.coordsSexaToDeg(" ".join(df["RA_(J2000)"][i].split(":")+df["Dec"][i].split(":")))
  #df["ra"][i]=np.round(df["ra"][i], decimals=3)
  #df["dec"][i]=np.round(df["dec"][i], decimals=3)
  if i%500==0:
    print(i)

In [0]:
#for i in range(len(df)):
#  df["ra"][i]=float("%.6f" %np.round(df["ra"][i], decimals=6))
#  df["dec"][i]=float("%.6f" %np.round(df["dec"][i], decimals=6))

In [0]:
df.to_csv("/gdrive/My Drive/Colab Notebooks/Catalina_Vars.csv")

In [0]:
df = pd.read_csv("/gdrive/My Drive/Colab Notebooks/Catalina_Vars.csv")
df1 = pd.read_table("/gdrive/My Drive/Colab Notebooks/DR1_sci_data.txt", delim_whitespace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  


In [0]:
# extracting RA and Dec columns from CRTS and ZTF data. 
# Here ZTF data used is http://web.ipac.caltech.edu/staff/fmasci/ztf/dr1/DR1_sci_data.txt.gz
#ztf_ra=np.round(np.array(df1["CenterR.A.[deg]"]),6)
#ztf_dec=np.round(np.array(df1["CenterDec.[deg]"]),6)

ztf_ra=np.array(df1["CenterR.A.[deg]"])
ztf_dec=np.array(df1["CenterDec.[deg]"])

crts_ra=np.array(df["ra"])
crts_dec=np.array(df["dec"])

In [0]:
# constructing a list of tuples as (RA, Dec)
#ztf_radec=[(ztf_ra[j],ztf_dec[j]) for j in range(len(ztf_ra))]
#crts_radec=[(crts_ra[j],crts_dec[j]) for j in range(len(crts_ra))]

In [0]:
# finding the intersection of the sets
#set(ztf_radec).intersection(set(crts_radec))

set()

In [0]:
#ra,ra_ztf,ra_crts=np.intersect1d(ztf_ra,crts_ra, return_indices=True)
#dec,dec_ztf,dec_crts=np.intersect1d(ztf_dec,crts_dec, return_indices=True)

In [0]:
#matched_indices=np.intersect1d(ra_ztf,dec_ztf)

In [0]:
#matched_indices

In [0]:
#def multidim_intersect(arr1, arr2):
#    arr1_view = arr1.view([('',arr1.dtype)]*arr1.shape[1])
#    arr2_view = arr2.view([('',arr2.dtype)]*arr2.shape[1])
#    intersected = np.intersect1d(arr1_view, arr2_view)
#    return intersected.view(arr1.dtype).reshape(-1, arr1.shape[1])

In [0]:
#multidim_intersect(ztf_radec,crts_radec)

array([[227.196,  -3.169]])

In [0]:
from astropy.coordinates import SkyCoord
from astropy import units as u
crts = SkyCoord(ra=crts_ra*u.degree, dec=crts_dec*u.degree)  
ztf = SkyCoord(ra=ztf_ra*u.degree, dec=ztf_dec*u.degree)  
idx, d2d, d3d = crts.match_to_catalog_sky(ztf)

In [0]:
matches=ztf[idx]

In [0]:
max_sep = 3.0# * u.arcsec 
idx_c, d2d_c, d3d_c = crts.match_to_catalog_3d(ztf) 
sep_constraint = d3d_c < max_sep 
crts_matches = crts[sep_constraint] 
ztf_matches = ztf[idx_c[sep_constraint]] 

In [0]:
np.in1d(ztf_matches,matches)

array([False, False, False, ..., False, False, False])

In [0]:
ztf_matches[0]

<SkyCoord (ICRS): (ra, dec) in deg
    (0.36852968, 10.40273978)>

In [0]:
matches[0]

<SkyCoord (ICRS): (ra, dec) in deg
    (0.36852968, 10.40273978)>

In [0]:
ztf_matches[0].ra.deg==matches[0].ra.deg

True

In [0]:
ztf_matches[4258].ra.deg==matches[4258].ra.deg

True

In [0]:
crts_matches.shape

(47055,)

In [0]:
len(set(crts_matches))

47055

In [0]:
crts_ra.shape

(47055,)

In [0]:
matches_ra=[]
for h in range(matches.shape[0]):
  matches_ra=matches_ra+[matches[h].ra.deg]
matches_ra=np.array(matches_ra)

In [0]:
matches_dec=[]
for h in range(matches.shape[0]):
  matches_dec=matches_dec+[matches[h].dec.deg]
matches_dec=np.array(matches_dec)

In [0]:
#ztf_idx=np.intersect1d(ztf_ra,matches_ra,return_indices=True)[2]

In [0]:
ztf_idx_ra=np.arange(ztf_ra.shape[0])[np.in1d(ztf_ra, matches_ra)]

In [0]:
ztf_idx_dec=np.arange(ztf_dec.shape[0])[np.in1d(ztf_dec, matches_dec)]

In [0]:
np.intersect1d((ztf_idx_dec==ztf_idx_ra),False)

array([], dtype=bool)

In [0]:
len(set(ztf_dec))==len(set(ztf_ra))

False

In [0]:
len(set(matches_dec))==len(set(matches_ra))

True

In [0]:
ztf_idx_ra

array([  15116,   15126,   19761, ..., 3400342, 3400359, 3409365])

In [0]:
df_ztf_ids=df1.take(ztf_idx_ra)

In [0]:
df_ztf_ids.to_csv("/gdrive/My Drive/Colab Notebooks/matched_ZTF_ids.csv")

In [0]:
#from astropy.table import QTable
#tbl = QTable()

In [0]:
#tbl["RA-Dec"]=matches

In [0]:
#from astropy.table import NdarrayMixin
#array=NdarrayMixin(matches)

In [0]:
#l=np.array([2,3,4,5,6,4])
#m=np.array([4,2,2])

In [0]:
#np.in1d(l,m)

array([ True, False,  True, False, False,  True])

In [0]:
#np.arange(l.shape[0])[np.in1d(l, m)]

array([0, 2, 5])

In [0]:
#np.intersect1d(l,m,return_indices=True)

(array([2, 4]), array([0, 2]), array([1, 0]))